### Примечание автора

Здесь опять же применяются операции, воспроизводимость которых не может быть гарантирована из-за обращения к геокодеру яндекса и к OSM, поэтому результат работы данных блоков в точности имеет путь ./data/prep/data.csv

Полученные по OSM данные также лежать в папке ./data/prep

In [5]:
import numpy as np, pandas as pd
from math import sin, cos, sqrt, atan2, radians
import json, re, gc, pickle, overpass

from tqdm import tqdm_notebook
from sklearn.neighbors import NearestNeighbors

R = 6373.0 # радиус земли в километрах

def distance(x,y):
    lat_a, long_a, lat_b, long_b = map(radians, [*x,*y])    
    dlon = long_b - long_a
    dlat = lat_b - lat_a
    a = sin(dlat / 2.)**2 + cos(lat_a) * cos(lat_b) * sin(dlon / 2.)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

### Считывание исходных данных

In [2]:
train = pd.read_csv("./data/init_train.csv")
test = pd.read_csv("./data/init_test.csv")

for df in [train, test]:
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.rename({'atm_group':'group'}, axis=1, inplace=True)
    
train['isTrain'] = True
test['isTrain'] = False

data = train.append(test, sort=False).reset_index(drop=True)
data['loc_null'] = data[['lat', 'long']].isnull().any(axis=1).astype(int)
firms_ids = data[['lat', 'long']].isnull().any(axis=1)

### Нахождение пропущенных геолокаций с помощью Яндекса

In [4]:
from yandex_geocoder import Client

coords = dict()  # (longitude, latitude)
errors = []
for row in tqdm_notebook(data[firms_ids]['address'].values, leave=False):
    try: 
        coords[row] = Client.coordinates(row) # returns (longitude, latitude)
    except Exception as e:
        errors.append(row)
        
def get_lat(x):
    if x not in coords:
        return np.nan
    else:
        return coords[x][1]
    
def get_long(x):
    if x not in coords:
        return np.nan
    else:
        return coords[x][0]
    
data.loc[firms_ids, 'lat'] = data[firms_ids]['address'].map(lambda x: get_lat(x))
data.loc[firms_ids, 'long'] = data[firms_ids]['address'].map(lambda x: get_long(x))

data[['lat', 'long']] = data[['lat', 'long']].astype(float)

HBox(children=(IntProgress(value=0, max=420), HTML(value='')))

### Неправильно найденные локации и отсутствующие локации

Вбиваем ручками

In [7]:
text2coord = { 
    'ABB 6B NAB-CHELNINSKIJ    NAB.CHELNY  ': (55.708725, 52.357388), #(широта, долгота)
    'D. 23, KORP. 1, UL. MATRO BIYSK G     ': (52.497357, 85.146107),
    'D. 33, PR-KT KOMMUNISTICH YUZHNO-SAKHA': (46.959107, 142.741459),
    'ABB 111 KRASNOARMEJSK.    JOSHKAR-OLA ': (56.644487, 47.858328),
    '49 CHERNOISTOCHENSKOE H/W NIZHNIY TAGI': (57.869815, 59.942484),
    'ABB 158 KRASNOKOKSH-AYA   KAZAN       ': (55.818886, 49.059734),
    "D. 118/11, UL. BOL'SHAYA  YAROSLAVL G ": (57.621632, 39.861428),
    'G. ELEC, UL. RADIOT       ELEC        ': (55.781888, 38.443663),
    
    'D. 19, LIT. V, PER. 4-I V PARGOLOVO P ': (60.077258, 30.249680),
    'SOVETSKAYA 25             S. MAYA     ': (57.813974, 28.333614),
    'V/CH. 06987               ENGELS G    ': (51.487954, 46.207807),
    'D. 22, PR-KT MASHINOSTROI YAROSLAVL G ': (57.649540, 39.948891),
    
    '133A MOZHAYSKOE H/W.      MOSCOW      ': (55.686902, 37.296845),
    'UNKNOWN                   DZERZHINSK G': (56.240710, 43.464212),
    'D. 76, PR-KT TRAKTOROSTRO CHEBOXARY G ': (56.109195, 47.330258),
    "D. 133, UL. BRAT'EV KASHI CHELYABINSK ": (55.172154, 61.305321),
    "D. 30, UL. BOL'SHAYA MORS SANKT-PETERB": (59.933873, 30.312974),
    "D. 2, UL. KORABEL'NAYA NA VLADIVOSTOK ": (43.114493, 131.886231),
    "D. 88, PR-KT ZOI KOSMODEM AZOV G      ": (47.100637, 39.419676),
    'D. 123, UL. KRASNOARMEISK BLAGOVESHCHE': (50.269183, 127.539301),
    'D. 29, NAB. SEREBRYANICHE MOSKVA G    ': (55.750581, 37.652528),
    "D. 142V, UL. RASKOL'NIKOV SARAPUL G   ": (56.476519, 53.815355),
}

for key in text2coord:
    data.loc[data['address'] == key, 'lat'] = text2coord[key][0]
    data.loc[data['address'] == key, 'long'] = text2coord[key][1]
    
data['coord'] = data[['lat', 'long']].apply(lambda x: (x['lat'], x['long']), axis=1)

coord2idx = {j: i for i, j in enumerate(data['coord'].unique())}
data['coord_idx'] = data['coord'].map(coord2idx)

In [ ]:
data.to_csv("./data/prep/data.csv", index=False)

### Окрестности точек выборки из OpenStreetMaps

In [ ]:
# заняло 1 час 17 минут 17 секунд

def get_info(lat, long, delta=0.01): # returns points with (longitude, latitude)
    info = api.get('node(' + str(lat - delta) + ',' + str(long - delta) \
                    + ',' + str(lat + delta) + ',' + str(long + delta) + ')')
    return [(node['geometry']['coordinates'], node['properties']) 
            for node in info['features'] if len(node['properties']) > 0]

api = overpass.API(endpoint='https://overpass.kumi.systems/api/interpreter', timeout=1000)

DELTA = 0.005
errors = []
osm_data = dict()
for row in tqdm_notebook(data[['coord', 'coord_idx']].drop_duplicates().values):
    lat = row[0][0]
    lon = row[0][1]
    try:
        tmp = get_info(lat, lon, delta=DELTA)
        tmp = [(node[0], node[1], distance([lat, lon], [node[0][1], node[0][0]])) 
                         for node in tmp]
        osm_data[row[1]] = sorted(tmp, key = lambda x: x[2])
    except Exception as e:
        errors.append(row)

with open('./data/prep/osm_data_0.005.pickle', 'wb') as fout:
    pickle.dump(osm_data, fout, protocol=pickle.HIGHEST_PROTOCOL)